In [9]:
// Source of the Data
//http://archive.ics.uci.edu/ml/datasets/Wholesale+customers

// info on the data:
// 1)FRESH: annual spending (m.u.) on fresh products (Continuous);
// 2)MILK: annual spending (m.u.) on milk products (Continuous);
// 3)GROCERY: annual spending (m.u.)on grocery products (Continuous);
// 4)FROZEN: annual spending (m.u.)on frozen products (Continuous)
// 5)DETERGENTS_PAPER: annual spending (m.u.) on detergents and paper products (Continuous)
// 6)DELICATESSEN: annual spending (m.u.)on and delicatessen products (Continuous);

In [10]:
// To see less warnings
import org.apache.log4j._
Logger.getLogger("org").setLevel(Level.ERROR)

// Import SparkSession
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
                        .appName("Kmean")
                        .config("spark.master", "local")
                        .getOrCreate()

import spark.implicits._

import org.apache.log4j._
import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2562db86
import spark.implicits._


In [11]:
// Import Kmeans clustering Algorithm
import org.apache.spark.ml.clustering.KMeans

// Load the Wholesale Customers Data
val dataset = spark.read.option("header","true").option("inferSchema","true").csv("Wholesale customers data.csv")

import org.apache.spark.ml.clustering.KMeans
dataset: org.apache.spark.sql.DataFrame = [Channel: int, Region: int ... 6 more fields]


In [12]:

// Select the following columns for the training set:
// Fresh, Milk, Grocery, Frozen, Detergents_Paper, Delicassen
// Cal this new subset feature_data
val feature_data = dataset.select($"Fresh", $"Milk", $"Grocery", $"Frozen", $"Detergents_Paper", $"Delicassen")


// Import VectorAssembler and Vectors
import org.apache.spark.ml.feature.{VectorAssembler,StringIndexer,VectorIndexer,OneHotEncoder}
import org.apache.spark.ml.linalg.Vectors

// Create a new VectorAssembler object called assembler for the feature
// columns as the input Set the output column to be called features
// Remember there is no Label column
val assembler = new VectorAssembler().setInputCols(Array("Fresh", "Milk", "Grocery", "Frozen", "Detergents_Paper", "Delicassen")).setOutputCol("features")

// Use the assembler object to transform the feature_data
// Call this new data training_data
val training_data = assembler.transform(feature_data).select("features")

// Create a Kmeans Model with K=3
val kmeans = new KMeans().setK(3).setSeed(1L)

// Fit that model to the training_data
val model = kmeans.fit(training_data)



feature_data: org.apache.spark.sql.DataFrame = [Fresh: int, Milk: int ... 4 more fields]
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, VectorIndexer, OneHotEncoder}
import org.apache.spark.ml.linalg.Vectors
assembler: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_b76289929921, handleInvalid=error, numInputCols=6
training_data: org.apache.spark.sql.DataFrame = [features: vector]
kmeans: org.apache.spark.ml.clustering.KMeans = kmeans_4effe0af2844
model: org.apache.spark.ml.clustering.KMeansModel = KMeansModel: uid=kmeans_4effe0af2844, k=3, distanceMeasure=euclidean, numFeatures=6


In [13]:
// Shows the result.
println("Cluster Centers: ")
model.clusterCenters.foreach(println)


Cluster Centers: 
[7390.958456973294,4439.768545994066,6292.19584569733,2495.53412462908,2238.6528189910982,1158.4480712166173]
[32768.013333333336,4827.68,5723.146666666667,5535.92,1074.1200000000001,2066.6400000000003]
[11849.17857142857,24717.10714285714,33887.71428571428,3409.3214285714284,15459.714285714284,4483.857142857142]
